In [ ]:
from wrangle import *


In [ ]:
programs = programs_dict()


In [ ]:
all= programs["All"]

In [ ]:

ds = programs["DS"]
full_stack = programs["Full_Stack_PHP"]
front_end = programs["Front_End"]

# Questions:


## 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?


In [ ]:
# Question 1
# 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?


ds_q1 = q1(ds)
ds_q1

In [ ]:

full_stack_q1 = q1(full_stack)
full_stack_q1


In [ ]:

front_end_q1 = q1(front_end)
front_end_q1
# since front end is such a small number I will not use them going forward


---
<br>

---

## 2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?


In [ ]:
# Question 2
# 2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

ds_q2 = question2(ds)
ds_q2


In [ ]:
full_stack_q2 = question2(full_stack[full_stack.name != "Staff"])
full_stack_q2


---
<br>

---

## 3. Are there students who, when active, hardly access the curriculum?
## If so, what information do you have about these students?


In [ ]:
# Question 3
# 3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?


question3(ds)


In [ ]:

question3(full_stack[full_stack.name != "Staff"])


## Yes,we can see the students and respective cohorts they belong to. 
## They we identified by picking the bottom 25% of access counts with respect to program type.
---
<br>

---

## 4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP ## 

In [ ]:
# Question 4
# Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? 
# Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?


q4(ds)

q4(full_stack[full_stack.name != "Staff"])




###  These ids are suspicious because they have very high number of ips associated with the same id. 
###  They are above the 95th percentile.
###   Due to this high number of IP's it is likely that they are web-scraping
### We will explore the question of if they shouldn't be in the next question.

---
<br>

---


## 5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. 
## Do you see any evidence of that happening?
## Did it happen before?

In [ ]:
# 5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off.
# Do you see any evidence of that happening? Did it happen before?





In [ ]:
q5(ds,full_stack)



### I found the common intersection between all DS cohorts. 
### Using that set I was able to flag all the students in Web dev who were accessing DS materials. 
### This set was rather conservative so there could in fact be more, nonetheless it clearly shows that access has not been entirely revoked only reduced, and there have been relatively high access past the point in 2019 where access was supposed end.
---
<br>

---

## 6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?


In [ ]:
#6. What topics are grads continuing to reference after graduation and into their jobs (for each program)

q6_ds=q6(ds)
q6_ds



In [ ]:

q6_full_stack=q6(full_stack[full_stack.name!='Staff'])
q6_full_stack

### We answer this question by presenting the lists above.
---
<br>

---

## 7. Which lessons are least accessed?


### We answer the two questions below by selecting lessons accessed the least in Data science and in Web Dev, however since there are multiple sites only accessed once we have a long list for each program.

In [ ]:
#7 Which lessons are least accessed?

q7_ds=q7(ds)
q7_ds



In [ ]:

q7_full_stack=q7(full_stack[full_stack.name!='Staff'])
q7_full_stack



As a bonus we start to play with Density based scanning for anomaly detection. 

In [ ]:


import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler,LabelEncoder


# Generate random dataset with a DateTime index and discrete variables
# dates = pd.date_range(start='2022-01-01', end='2022-01-31', freq='D')
ds_new=ds.set_index(ds.datetime, drop=True).drop(columns="datetime")
# Create an instance of LabelEncoder
le = LabelEncoder()

# Apply LabelEncoder to all columns of type 'object' in the DataFrame
ds_new=ds_new.drop(columns=['start_date','end_date','lesson'])
ds_new = ds_new.apply(lambda x: le.fit_transform(x) if x.dtype == 'object' else x)

X = ds_new


# Scale the data
X_scaled = StandardScaler().fit_transform(X)
# Apply DBSCAN clustering algorithm to detect anomalies
# A rule to use is that min sample should be greater than or equal to the number of dimensions+1
min_samples=len(X.columns)+1



dbscan = DBSCAN(eps=0.5, min_samples=min_samples)
dbscan.fit(X_scaled)

# Get the labels assigned to each data point
labels = dbscan.labels_

# Find the indices of anomalies (i.e., points labeled as -1)
anomaly_indices = np.where(labels == -1)[0]

# Print the number of anomalies detected
print(f"Number of anomalies detected: {len(anomaly_indices)}")

# Plot the dataset with anomalies highlighted in red
plt.scatter(X_scaled[:, 0], X_scaled[:, 1], c=labels, cmap='cool')
plt.scatter(X_scaled[anomaly_indices, 0], X_scaled[anomaly_indices, 1], c='r', marker='x', label='Anomalies')
plt.legend()
plt.show()


original_labels = X.index[anomaly_indices]

ds_new=ds.set_index(ds.datetime, drop=True).drop(columns=['start_date','end_date','lesson','datetime'])

ds_anomalies=ds_new.loc[original_labels]

#compare the percentage of anomalies based on the original dataframe.
print(f'We find our anomalies to be {(len(ds_anomalies)/len(ds_new))*100:.4f}% of our original dataframe.')

print( "We groupby id and use the boolean cohort_window column to see the percentage access a user had within their cohort time frame and the overall count of their access.  Given this information, I assume these users dropped out of the program, and DB SCAN was capturing those users as anomalies.\n\n")
db_scan_analysis=ds_anomalies.groupby('id').cohort_window.agg(['mean','count']).sort_values(by='count',ascending=False)
display(db_scan_analysis.head(),db_scan_analysis.describe())

Executive Summary:

All questions were answered. Yet as bonus I employed DB SCAN. 

DB SCAN proved to be an interesting feature but it still needs to be further inspected to show why something is an anomaly. I could see this being an interesting tool to trigger warnings, however as I have it currently implemented it seemed to highlight low access and ignored high volume users and those who accessed outside of their cohort date, which seem more relevant given our use case.

I also only used it once for the subset user logs where the users were DataScience students. 